# Generate Questions and Errors from Biblical Data
I used GPT to create a prompt for me.  See this here https://chat.openai.com/share/37fce0d6-a384-4181-a2bf-6b2a18fd96bb



First, set your `OPENAI_API_KEY` environment variable. This allows you to securely authenticate with the OpenAI API, without worrying about leaving the value in your code. You can find your API key in your [OpenAI dashboard](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key).

In [12]:
import getpass, os, openai

secret_key = getpass.getpass('Enter OpenAI secret key: ') 
os.environ['OPENAI_API_KEY'] = secret_key

MODEL = 'gpt-3.5-turbo'





In [16]:
# If using the azure API endpoint, use the Azure instead
if not secret_key.startswith('sk-'):
    print("Using Azure API endpoint")
    openai.api_type = "azure"
    openai.api_base = os.environ['OPENAI_API_BASE'] = 'https://americasopenai.azure-api.net/'
    openai.api_version = os.environ['OPENAI_API_VERSION'] = '2023-05-15'
    openai.api_key = secret_key
    
    MODEL = 'gpt-35-turbo-16k'


Using Azure API endpoint


## Generate questions and answers from biblical data
Have GPT take on a role of linguistical biblical expert


In [7]:
examples = """
[
  {
    "verse": "Genesis 1:1",
    "question": "Who created the heavens and the earth?",
    "answer": "God",
    "examples": {
      "bad_translation": "At the start, gods made sky and rocks.",
      "wrong_answer": "gods"
    }
  },
  {
    "verse": "Genesis 1:1",
    "question": "What did God create in the beginning?",
    "answer": "The heavens and the earth",
    "examples": {
      "bad_translation": "In the outset, divinities fashioned the atmosphere and stones.",
      "wrong_answer": "atmosphere and stones"
    }
  },
  {
    "verse": "Genesis 1:1",
    "question": "What time frame is indicated for the creation?",
    "answer": "In the beginning",
    "examples": {
      "bad_translation": "Initially, a deity brought forth the skies and the land.",
      "wrong_answer": "Initially"
    }
  }
]
"""

In [18]:
import json
def query(verse="Jn 3:16"):
    messages = [
        {"role":"system","content": "Generate a structured JSON array that includes quality control questions aimed at identifying potential errors in translations of the following Bible verses: Genesis 1:1, John 3:16, and Psalm 23:1. The JSON should include the verse reference, the question, the correct answer, and examples of bad translations with the wrong answers they would yield."},
        {"role":"user","content": "Matthew 28:19"},
        {"role":"system","content": examples},
    ]
    response = openai.ChatCompletion.create(
            engine=MODEL,
            messages = messages,
            temperature=0.25,
            max_tokens=8000,
            stop=None)
    result = response.get('choices',[{}])[0].get('message',{'content':''}).get('content','') 
    try:
        return json.loads(result)
    except:
        # extract json out of the result string
        result = result[result.find('{'):result.rfind('}')+1]
        return json.loads(result)

query("Jn 1:1")

[{'verse': 'Genesis 1:1',
  'question': 'Who created the heavens and the earth?',
  'answer': 'God',
  'examples': {'bad_translation': 'At the start, gods made sky and rocks.',
   'wrong_answer': 'gods'}},
 {'verse': 'Genesis 1:1',
  'question': 'What did God create in the beginning?',
  'answer': 'The heavens and the earth',
  'examples': {'bad_translation': 'In the outset, divinities fashioned the atmosphere and stones.',
   'wrong_answer': 'atmosphere and stones'}},
 {'verse': 'Genesis 1:1',
  'question': 'What time frame is indicated for the creation?',
  'answer': 'In the beginning',
  'examples': {'bad_translation': 'Initially, a deity brought forth the skies and the land.',
   'wrong_answer': 'Initially'}},
 {'verse': 'John 3:16',
  'question': 'Who did God give his one and only Son for?',
  'answer': 'The world',
  'examples': {'bad_translation': 'God gave his unique Son for the people.',
   'wrong_answer': 'the people'}},
 {'verse': 'John 3:16',
  'question': 'What did God giv